# Measurement-induced dephasing on Starmon-5

In [1]:
import time
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from typing import List
import itertools as it

import qiskit
import qiskit.tools.jupyter
from qiskit import QuantumCircuit
import qiskit.circuit.library.standard_gates as gates
from qiskit.tools.visualization import circuit_drawer, plot_histogram

from alive_progress import alive_it, alive_bar
from tqdm import tqdm

# import pyperclip as clip
from importlib import reload
import inspire_experiments
reload(inspire_experiments)
from inspire_experiments import ExperimentData, get_starmon_status, inspire_login, get_file_header, measure_readout_correction

# %matplotlib notebook
%matplotlib inline
%qiskit_version_table

In [2]:
api, backend = inspire_login()
backend.status()

Backend status:  EXECUTION


In [7]:
get_starmon_status(api)

'OFFLINE'

## Measurements

In [4]:
def measure_msmt_induced_dephasing(
        measured_qb: int,
        tested_qb: int = None,
        tested_qb_state: str = '',
        tested_qb_msmt: bool = True,
        dd_sequence: str = 'ID',
        do_tomography: bool = True,
        tomography_on_equator: bool = True,
        exp_basename: str = "msmt_induced_dephasing",
        extra_label: str = None,
        show_circuit: bool = False,
        send_jobs: bool = True
):
    jobs = []
    circuits = []
    if do_tomography:
        # tomo_gates = {'X': ('ry', -np.pi/2), 'Y': ('rx', np.pi/2), 'Z': ('id',  None)}
        tomo_gates = {'X': gates.RYGate(-np.pi/2), 'Y': gates.RXGate(np.pi/2), 'Z': gates.IGate()}
    else:
        # tomo_gates = {'Z': ('id', None)}
        tomo_gates = {'Z': gates.IGate()}

    # bar = alive_it(tomo_gates.items(), total=len(tomo_gates.items()))
    bar = tqdm(tomo_gates.items(), total=len(tomo_gates.items()))
    for tomo_option, gate in bar:
        circuit = QuantumCircuit(5,5)
        circuit.rx(np.pi/2, measured_qb)
        if tested_qb is not None: # can't autocast here, qb can be 0!
            if tested_qb_state == '' or tested_qb_state == '0':
                circuit.id(tested_qb)
            elif tested_qb_state == '1':
                circuit.x(tested_qb)
            elif tested_qb_state == '+':
                circuit.ry(np.pi/2, tested_qb)
            else:
                raise ValueError(f"State of tested qubit {tested_qb_state} unknown!")
        circuit.barrier(range(5))
        if tested_qb is not None and tested_qb_msmt:
            circuit.measure(tested_qb, tested_qb)

        if dd_sequence.upper() == 'XY4':
                # add decoupling sequence XY4, padded with 2 IDs at beginning and end
                # to fit into exactly 100 cycles (= 2 mus = msmt time)
                for _ in range(2):
                    circuit.id(measured_qb)
                # for _ in range(1):
                for _ in range(100//8):
                    for _ in range(4):
                        circuit.x(measured_qb)
                        circuit.y(measured_qb)
                for _ in range(2):
                    circuit.id(measured_qb)
                circuit.barrier(range(5))

        elif dd_sequence.upper() == 'ID':
            # 100 x 20ns ID gate = 2mus = msmt length
            for _ in range(100):
                circuit.id(measured_qb)
            circuit.barrier(range(5))
        else:
            raise ValueError(f"Unknown sequence {dd_sequence} given!")

        circuit.barrier(range(5))
        if not tomography_on_equator:
            circuit.rx(np.pi/2, measured_qb)
            circuit.barrier(range(5))
        # getattr(circuit, gate)(angle, measured_qb) if angle is not None else getattr(circuit, gate)(measured_qb)

        circuit.append(gate, [measured_qb])
        circuit.measure(measured_qb, measured_qb)
        circuits += [circuit]

        if show_circuit:
            display(circuit.draw(output='mpl', scale=1, fold=30))

        exp_name = exp_basename + f"_qbs{measured_qb, tested_qb}"
        exp_name += f"_testedQb{tested_qb_state}" if tested_qb else ''
        exp_name += f"_testedQbMsmt{int(tested_qb_msmt)}"
        exp_name += "_eqTomo" if tomography_on_equator else ''
        exp_name += f"_{dd_sequence}" if dd_sequence else ''
        exp_name += extra_label if extra_label else ''
        exp_name += f"_M{tomo_option}" if do_tomography else ''
        print(exp_name)

        if send_jobs:
            header = get_file_header(circuit)
            print("Backend status: ", get_starmon_status(api))
            job = qiskit.execute(circuit, shots=2**14, optimization_level=0, backend=backend)
            print(f"Job with ID {job.job_id()} got status {job.status()}")
            ExperimentData.save_job_result(job, exp_name, header)
            jobs += [job]

    return jobs, circuits

## Measure all pairs

In [ ]:
# pairs = [(2,1), (2,3), (3,2), (2, None), (3, None)]
# pairs = [ (3,2), (2, None), (3, None)]
# states = ['0', '1', '+']
# pairs = [(2, None), (3, None)]
pairs = [(3,None), (3,2)]
states= ['0',]

with alive_bar() as bar:
    for pair in pairs:
        for state in states:
            print(get_starmon_status(api))
            if pair[1] is None:
                measure_msmt_induced_dephasing(measured_qb=pair[0], dd_sequence='ID', tested_qb_state='', send_jobs=False, show_circuit=True)
                measure_readout_correction(qubits=[pair[0]], extra_label=f"_testedQb{state}", backend=backend, send_jobs=False, show_circuit=False)
                break
            else:
                measure_msmt_induced_dephasing(*pair, dd_sequence='ID', tested_qb_state=state, send_jobs=False, show_circuit=True)
            bar()

## Measure all combinations of MID cases

In [5]:

# pairs = [(3,2), (4,2)]
pairs = [(2,4)]
tested_qb_states = ['0', '1']
measured_qb_dd = ['ID', 'XY4']
tested_qb_msmt = [True, False]


# with alive_bar(total=8*len(pairs)) as bar:
for pair in pairs:
    measure_readout_correction(qubits=[pair[0]], extra_label=f"", backend=backend, send_jobs=True, show_circuit=False)
    for state in tested_qb_states:
        for msmt in tested_qb_msmt:
            for dd in measured_qb_dd:
                print(get_starmon_status(api))
                measure_msmt_induced_dephasing(*pair, dd_sequence=dd, tested_qb_msmt=msmt, tested_qb_state=state, send_jobs=True, show_circuit=False)
                # bar()


  0%|          | 0/2 [00:00<?, ?it/s]13:52:24 - inspire_experiments.experiment_library:INFO - readout_correction_qbs[2]_state0
13:52:24 - inspire_experiments.experiment_library:INFO - Backend state: EXECUTION
13:52:24 - inspire_experiments.experiment_library:INFO - Measuring readout correction: state ('0',)
13:52:24 - inspire_experiments.data_utils:INFO - Saving results for job 7168618 in ./data/
 50%|█████     | 1/2 [00:57<00:57, 57.21s/it]13:53:21 - inspire_experiments.experiment_library:INFO - readout_correction_qbs[2]_state1
13:53:21 - inspire_experiments.experiment_library:INFO - Backend state: IDLE
13:53:21 - inspire_experiments.experiment_library:INFO - Measuring readout correction: state ('1',)
13:53:22 - inspire_experiments.data_utils:INFO - Saving results for job 7168620 in ./data/
100%|██████████| 2/2 [02:17<00:00, 68.76s/it]


IDLE


  0%|          | 0/3 [00:00<?, ?it/s]

msmt_induced_dephasing_qbs(2, 4)_testedQb0_testedQbMsmt1_eqTomo_ID_MX
Backend status:  IDLE


13:54:42 - inspire_experiments.data_utils:INFO - Saving results for job 7168622 in ./data/


...

 67%|██████▋   | 2/3 [02:42<01:21, 81.05s/it]

msmt_induced_dephasing_qbs(2, 4)_testedQb1_testedQbMsmt0_eqTomo_XY4_MZ
Backend status:  IDLE


14:26:01 - inspire_experiments.data_utils:INFO - Saving results for job 7168668 in ./data/


Job with ID 7168668 got status JobStatus.QUEUED


100%|██████████| 3/3 [04:02<00:00, 80.97s/it]


In [6]:

backend._QuantumInspireBackend__api.get_jobs()



[OrderedDict([('url', 'https://api.quantum-inspire.com/jobs/7208771/'),
              ('name', 'circuit-130'),
              ('id', 7208771),
              ('status', 'COMPLETE'),
              ('input', 'https://api.quantum-inspire.com/assets/7282393/'),
              ('backend', 'https://api.quantum-inspire.com/backends/8/'),
              ('backend_type',
               'https://api.quantum-inspire.com/backendtypes/11/'),
              ('results',
               'https://api.quantum-inspire.com/jobs/7208771/result/'),
              ('queued_at', '2022-08-30T12:26:01.476690Z'),
              ('number_of_shots', 16384),
              ('full_state_projection', False),
              ('user_data',
               '{"name": "circuit-130", "memory_slots": 5, "creg_sizes": [["c", 5]], "measurements": {"measurements_state": [[2, 2]], "measurements_reg": [[2, 2]], "number_of_qubits": 5, "number_of_clbits": 5}}')]),
 OrderedDict([('url', 'https://api.quantum-inspire.com/jobs/7208769/'),
       